In [ ]:
"""20.1 ORM что это и зачем

ORM это Object-relational mapping, что означает «Объектно-реляционное отображение».
ORM это «технология программирования, которая связывает базы данных с концепциями объектно-ориентированных языков программирования» 
Проще говоря, ORM это некая прослойка между базой данных и серверным кодом, который позволяет нам работать с данными в терминах классов,
а не напрямую с таблицами БД.

И наконец самая простая формулировка.
ORM избавит нас от работы с чистым SQL-кодом, который ужасно раздувает наш проект и делает код не читабельным.
Вместо этого, мы просто опишем наши таблицы в классах python-на, с помощью которых, будем взаимодействовать с БД.
Также можно создавать, изменять, удалять, фильтровать и, самое главное, наследовать объекты классов, сопоставленные с таблицами БД,
что существенно сокращает наполнение кодовой базы.

Сравним объем и сложность кода при запросе сущности из БД напрямую из кода, и с использованием ORM. 

Код с чистым SQL
Мы подключаемся к базе данных sqlite, пишем запрос на создание таблицы,
создаем транзакцию, выполняем SQL-запрос и сохраняем наши изменения.
После этого мы пишем запрос на получение данных из таблицы по фильтру, выполняем его и завершаем транзакцию.

"""
import sqlite3

sqlite_connection = sqlite3.connect('sqlite_python.db')

create_table_query = '''CREATE TABLE players (
                       id INTEGER PRIMARY KEY,
                       name TEXT NOT NULL,
                       email TEXT NOT NULL UNIQUE,
                       joining_date datetime,
                       salary REAL NOT NULL);'''
cursor = sqlite_connection.cursor()
cursor.execute(create_table_query)
sqlite_connection.commit()

select_players_query = '''
SELECT * FROM players WHERE name = "Nikita";'''

cursor = sqlite_connection.cursor()
students = cursor.execute(select_players_query)
cursor.close()


"""Код с ORM
Мы описываем подключение к базе, создаем сессию, с помощью которой, в дальнейшем будем взаимодействовать с базой,
описываем класс для нашей таблицы, выполняем функцию создания таблицы 
и одной строчкой выполняем запрос на получение данных по фильтру.
"""

from sqlalchemy import Column, Integer, String, DateTime, Float
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import create_engine

engine = create_engine('sqlite:///sqlite_python.db')
Base = declarative_base()

Session = sessionmaker(bind=engine)
session = Session()


class Player(Base):
   __tablename__ = 'players'

   id = Column(Integer, primary_key=True)
   name = Column(String, nullable=False)
   email = Column(String, nullable=False, unique=True)
   joining_date = Column(DateTime)
   salary = Column(Float, nullable=False)


Base.metadata.create_all(engine)
players = session.query(Player).filter_by(name="Никита")


"""
Плюсы работы с ORM:

для реализации одинакового функционала требуется писать меньше кода; 
не нужно писать SQL-запросы;
не нужно самому создавать таблицы
такой код легко сопровождать


Минусы работы с ORM:
снижении скорости работы с базой
затраченное время на освоение ORM
потеря контроля над SQL-запросами
"""


"""Ключевой недостаток ORM заключается в снижении скорости работы с базой.
Слой транзакций, выполняющий преобразования, может быть недостаточно эффективным,
поэтому при обработке большого количества данных, работа приложения будет медленнее, чем при использовании чистого SQL.

Плюс, разработчику необходимо потратить какое-то время, чтобы освоить работу с ORM.

Так же я бы выделил недостаток в работе ORM - это возможная потеря контроля над запросами.
Согласитесь, когда вы пишете запрос на чистом SQL, то вероятность понять,
где может быть ошибка выше, нежели, чем когда вы используете ORM-функции.
Особенно это касается сложных запросов. Поэтому всегда сравнивайте запросы через ORM с фактическими, которые выполняются в SQL профилировщике сервера.


 
"""